## TEXT MINING PROJECT - GENDER POLITICS - IORDACHE

In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm
tqdm.pandas()

/home/alice/.local/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
dataset = pd.read_csv("/home/alice/Desktop/text mining project/convocationsReportResult.csv", sep = ";")

In [3]:
dataset.head()

,id,convocationid,downloadtime,text
0,9555,14593,2020-03-06 19:56:43.847,A t t i Parlamentari - 3016 - Camera dei Dep...
1,9480,14494,2020-03-06 19:55:34.845,C A M E R A DEI DEPUTATI \nI. \nSEDUTA ANTIHE...
2,9481,14495,2020-03-06 19:55:34.984,Aiti Parlahentarì - 7 - Caniera dei Deputali ...
3,9482,14497,2020-03-06 19:55:36.212,Atti ' Parlamentari - 15 - Camera dei Deputati...
4,9483,14498,2020-03-06 19:55:36.343,Atti Parlamentari - 59 - Camera dei Deputali \...


We define a function called "ottieni testo".
We take the very same columns we have and ask the function to split the text (therefore, the variable x becomes the dataframe) whenever it finds "La seduta comincia", since by looking at the text we we found this recursive pattern. Normally after the "La seduta comincia" we have a space and a timestamp; it is in position "\n". Therefore, we need to remove everything that part (pos = test.find("\n"), whenever it finds that specific expression.
Moreover, we created two new columns called "testo_seduta" and "pulito": we need the frist in order to have the text of each session without everything that comes before it on the page; we need "pulito" column just as a boolean in order to make a count of how many sessions (documents) it cannot recognize through the pattern we just saw.

In [4]:
def ottieni_testo(x):
    col = ['id', 'convocationid', 'downloadtime', 'text', 'testo_seduta', 'pulito']
    splitted = x.text.split("La seduta comincia")

    if len(splitted) == 2:
        testo = splitted[1]
        pos = testo.find("\n")

        end = -1
        if 'PAGINA BIANCA' in testo:
            end = testo.find('PAGINA BIANCA')

        if end != -1:
            return pd.Series([x.id, x.convocationid, x.downloadtime, x.text, testo[pos + 1:end], True], index=col) 
        else:
            return pd.Series([x.id, x.convocationid, x.downloadtime, x.text, testo[pos + 1:], True], index=col) 
        
    else:
        return pd.Series([x.id, x.convocationid, x.downloadtime, x.text, x.text, False],index=col)

In [5]:
dataset = dataset.progress_apply(ottieni_testo, axis=1)

100%|██████████| 3736/3736 [00:03<00:00, 1058.50it/s]


In [6]:
dataset.head()

,id,convocationid,downloadtime,text,testo_seduta,pulito
0,9555,14593,2020-03-06 19:56:43.847,A t t i Parlamentari - 3016 - Camera dei Dep...,"CUADALUPI, Segrctnrio, legge i I pro- \ncesso ...",True
1,9480,14494,2020-03-06 19:55:34.845,C A M E R A DEI DEPUTATI \nI. \nSEDUTA ANTIHE...,Costituzione deU’UBcio provvisorio \ndi Presid...,True
2,9481,14495,2020-03-06 19:55:34.984,Aiti Parlahentarì - 7 - Caniera dei Deputali ...,Insediamento e discorsb del Presidente. \nPRES...,True
3,9482,14497,2020-03-06 19:55:36.212,Atti ' Parlamentari - 15 - Camera dei Deputati...,"RICCIO, Segretario, legge il processo ver- \nb...",True
4,9483,14498,2020-03-06 19:55:36.343,Atti Parlamentari - 59 - Camera dei Deputali \...,"SULLO, Segretario, leg3e il processo ver- \n(E...",True


In [7]:
#Let's count the booleans.
dataset.pulito.value_counts()

True     3579
False     157
Name: pulito, dtype: int64

In [8]:
#Let's check the sessions that it didn't recognize.
non_puliti = dataset[dataset['pulito']==False]

Now we try to extract the sessions.
Take a random example:

In [9]:
print(dataset.testo_seduta[2500])

PRESIDENTE. La seduta è aperta.
Invito il deputato segretario a dare
lettura del processo verbale della seduta
precedente.
RAFFAELLO VIGNALI, Segretario,
legge il processo verbale della seduta del
21 dicembre 2016.
PRESIDENTE. Se non vi sono osserva-
zioni, il processo verbale si intende ap-
provato.
(È approvato).
Missioni.
PRESIDENTE. Comunico che, ai sensi
dell’articolo 46, comma 2, del Regola-
mento, i deputati Boschi, Fontanelli, Lo-
sacco e Orlando sono in missione a de-
correre dalla seduta odierna.
I deputati in missione sono complessi-
vamente nove, come risulta dall’elenco
depositato presso la Presidenza e che sarà
pubblicato nell’allegato A al resoconto
della seduta odierna (Ulteriori comunica-
zioni all’Assemblea saranno pubblicate nel-
l’allegato A al resoconto della seduta
odierna).
Annunzio della presentazione di disegni di
legge di conversione e loro assegna-
zione a Commissioni in sede referente
(ore 11,05).
PRESIDENTE. La Ministra per i rap-
porti con il Parlamento, c

#### Regex:

Round brakets contain the groups that will be matched: everything inside the brakets will be reported as a result. But, we don't want also the cases in which it only finds "\n"; for this reason, we write "?:" in the brakets, to indicate that we look for that group, but we don't write it in the result.

The "+" sign means find at least one occurence of everything that comes before.

"\b" is the word boundary; it indicates that the characters we are looking for need to be part of a word.

"[A-Z ]" is a range: it finds all the characters from the alphanumeric alphabet from A to Z (capital letters only!).
Note*: there is a space inside the square brakets; it means that we accept a word even if it is followed by a space. We need this, because after the brakets we have "+"; therefore, we accept more than just one word in capital letters followed by one another (ex: NAME SURNAME).


The regex finds for all the words in capital letters, repeated or not, preceded by "\n", because the interventions start on a new line like this:
NAME SURNAME intervention text etc

The result of this regex is a list of "match object" which has three elements in this case: the NAME SURNAME of each person speaking, the position in the original text where the NAME begins (the N of NAME)and the position in the original text where the SURNAME ends (the E of SURNAME).

In [10]:
import re

pattern = r"(?:\n)+(\b[A-Z ]+\b)"
l = re.finditer(pattern, dataset.testo_seduta[2500])

We see that sometimes we can find roman numbers ( I, III; XVII, etc). Our regex would recognize them as NAME SURNAME, but we don't need that.
Therefore, we use this (taken from): https://www.oreilly.com/library/view/regular-expressions-cookbook/9780596802837/ch06s09.html

In [11]:
def isRomanNumber(x):
    roman_result = re.match(r"^(?=[MDCLXVI])M*(C[MD]|D?C{0,3})(X[CL]|L?X{0,3})(I[XV]|V?I{0,3})$", x)
  
    if roman_result is not None:
        return True
    else:
        return False

`l` is an iterator which consumes when we use it (vuol dire che fatto un ciclo di for su `l`, per poterci lavorare ancora devo riottenere nuovamente `l`).
We therefore organize our results in a list like '[nome cognome, idx start, idx end]'.
Meanwhile, we filter the text by removing those results which consist only of "\n", a roman number, or any of the words we just list below:

In [12]:
custom_stopwords = ['LEGISLATURA', 'PAGINA BIANCA', 'STENOGRAFICO', 'STENOGRAFIA', 'TIPOGRAFIA']

results = []
for cacca in l:
    name = str(cacca.group(0)).strip()

    if name != '\\n' and (len(name) > 1) and not any(x in name for x in custom_stopwords) and not isRomanNumber(name):
        results.append([name, cacca.start(), cacca.end()])

There may be cases when during the split we got some capital letter words, but they are not the name of a deputy starting to talk (for example: they may be references or quotations of past laws or amendments).
These words come one next to the other: I have a list of words in a row and in the next line other words (still in capital letters) that I got in the results. By checking the original position of these words in the original text, I can remove them from the results and subsequently filter.

In [13]:
toRemove = []

for i, item in enumerate(results):
    if i+1 < len(results):
        if item[2] == results[i+1][1]:
            if i not in toRemove:
                toRemove.append(i)

            if i+1 not in toRemove:
                toRemove.append(i+1)

[results.pop(i) for i in sorted(toRemove, reverse=True)]

[['DOTT', 6863, 6868],
 ['ESTENSORE DEL PROCESSO VERBALE', 6832, 6863],
 ['DEL SERVIZIO RESOCONTI', 6809, 6832],
 ['IL CONSIGLIERE CAPO', 6789, 6809]]

In this way we obtained all the names of the speaking deputies. 
NB: in this case, ALFREIDER is quoted, but it is not a deputy who speaks. One possible next solution would be to download with SPARQL the names of the deputies of the assembly (in order to do that we need to resort to the id_session present in the other csv. From there we can obtain the reference to the assembly) and filter again: if we get a result that is not in the list of the deputies for that assembly, we remove it.
Next step: we get the index where a person starts talking; now we extract the text.

In [14]:
for item in results:
    print('%s: %02d-%02d' % (item[0], item[2], item[1]))

RAFFAELLO VIGNALI: 140-122
PRESIDENTE: 225-214
PRESIDENTE: 337-326
PRESIDENTE: 949-938
PRESIDENTE: 2542-2531
PRESIDENTE: 4854-4843
PRESIDENTE: 5238-5227
ALFREIDER: 6568-6557


In [15]:
text = dataset.testo_seduta[2500]

discorsi = []
for i, item in enumerate(results):
    if i+1 < len(results):
        discorso = text[item[2]:results[i+1][1]]
    elif i+1 == len(results):
        discorso = text[item[2]:]

    discorsi.append([item[0], discorso])

In [16]:
discorsi

[['RAFFAELLO VIGNALI',
  ', Segretario,\nlegge il processo verbale della seduta del\n21 dicembre 2016.'],
 ['PRESIDENTE',
  '. Se non vi sono osserva-\nzioni, il processo verbale si intende ap-\nprovato.\n(È approvato).\nMissioni.'],
 ['PRESIDENTE',
  '. Comunico che, ai sensi\ndell’articolo 46, comma 2, del Regola-\nmento, i deputati Boschi, Fontanelli, Lo-\nsacco e Orlando sono in missione a de-\ncorrere dalla seduta odierna.\nI deputati in missione sono complessi-\nvamente nove, come risulta dall’elenco\ndepositato presso la Presidenza e che sarà\npubblicato nell’allegato A al resoconto\ndella seduta odierna (Ulteriori comunica-\nzioni all’Assemblea saranno pubblicate nel-\nl’allegato A al resoconto della seduta\nodierna).\nAnnunzio della presentazione di disegni di\nlegge di conversione e loro assegna-\nzione a Commissioni in sede referente\n(ore 11,05).'],
 ['PRESIDENTE',
  '. La Ministra per i rap-\nporti con il Parlamento, con lettere in data\n30 dicembre 2016, ha presentato all

We apply now this reasoning on the whole dataset.

In [17]:
discorsi = []

def estrattore_discorso(x):
    pattern = r"(?:\n)+(\b[A-Z ]+\b)"
    l = re.finditer(pattern, x.testo_seduta)
  
    custom_stopwords = ['LEGISLATURA', 'PAGINA BIANCA', 'STENOGRAFICO', 'STENOGRAFIA', 'TIPOGRAFIA', 'DISCUSSIONI', 'SEDUTA', 'VOTAZIONI', 'DISCUS']
  
    results = []
    for match in l:
        name = str(match.group(0)).strip()

        if name != '\\n' and (len(name) > 1) and not any(x in name for x in custom_stopwords) and not isRomanNumber(name):
            results.append([name, match.start(), match.end()])

    toRemove = []
    for i, item in enumerate(results):
        if i+1 < len(results):
            if item[2] == results[i+1][1]:
                if i not in toRemove:
                    toRemove.append(i)
                if i+1 not in toRemove:
                    toRemove.append(i+1)

    [results.pop(i) for i in sorted(toRemove, reverse=True)]

    for i, item in enumerate(results):
        if i+1 < len(results):
            discorso = x.testo_seduta[item[2]:results[i+1][1]]
        elif i+1 == len(results):
            discorso = x.testo_seduta[item[2]:]

        discorsi.append([x.convocationid, item[0], discorso.strip()])

In [18]:
dataset.progress_apply(estrattore_discorso, axis=1)

100%|██████████| 3736/3736 [01:09<00:00, 53.86it/s] 


0       None
1       None
2       None
3       None
4       None
        ... 
3731    None
3732    None
3733    None
3734    None
3735    None
Length: 3736, dtype: object

In [19]:
df_discorsi = pd.DataFrame(discorsi, columns=['convocationid', 'deputato', 'discorso'])

In [20]:
df_discorsi.head(10)

,convocationid,deputato,discorso
0,14593,PRESIDENTE,. Con!unico che hanno chic- \nsto congedo i cl...
1,14593,PRESIDENTE,. L’ordine del giorno reca \nil seguito della ...
2,14593,VIVIANI LUCIANA,". Onorevole Mini- \nstro (mi rivolgo a lei, vi..."
3,14593,SCELBA,", Minis t ro dell’interno. Il Mini- \nstero d..."
4,14593,VIVIANI LUCIANA,". L a circolare, co- \nmunque, del 1944, fu ..."
5,14593,SCELBA,", Mini s t ro dell’interno. Non è \npiù in vi..."
6,14593,VIVIANI LUCIANA,. I1 passaggio è av- \nvenuto molto recentemen...
7,14593,C O S,"~ l’assistenza, ogni giorno piii, diventa \na..."
8,14593,GIUNTOLI GRAZTA,. L’abbiamo inter- \nrogata abbastanza! Ma qu...
9,14593,VIVIANI LUCTANA,". Bastava conside- \nrare, care colleghe, nel ..."


In [21]:
df_discorsi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 806960 entries, 0 to 806959
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   convocationid  806960 non-null  int64 
 1   deputato       806960 non-null  object
 2   discorso       806960 non-null  object
dtypes: int64(1), object(2)
memory usage: 18.5+ MB
